### Paso a paso para crear un Pipeline de procesamiento con Sklearn. Incluye un ejemplo de un transformador personalizado para escalar datos con log(x)

Para este ejemplo se utiliza el dataset de [California Housing Prices](https://www.kaggle.com/camnugent/california-housing-prices)

In [1]:
import pandas as pd
import numpy as np

# Se cargan los datos
housing = pd.read_csv('housing.csv')
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [2]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


### Separamos las caracteristicas de la variable target

In [3]:
features = housing.drop("median_house_value", axis=1)
labels = housing["median_house_value"].copy()

### Hacemos el train_test_split

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.30, random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(14448, 9) (14448,) (6192, 9) (6192,)


Se van a crear tres grupos de variables con el objetivo de aplicar transformaciones diferentes más adelante, por ejemplo:

* Se crea una variable continua que contiene `median_income` para más adelante aplicar una transformación logarítmica.
* Se crea una variable numérica que contine las siguientes features con el objetivo de aplicar otras transformaciones: 'longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households'.
* Se crea una variable categórica que contiene la variable `ocean_proximity`, para aplicar OneHotEncoder.

In [5]:
# Variable "continua" que contiene la  catacterística median_income
continua = X_train.loc[:,"median_income"]
continua_df = pd.DataFrame(continua)
continua_df.head(3)

,median_income
7061,4.1312
14689,2.8631
17323,4.2026


In [6]:
# Variable "numericas" que contiene todas las demás variables de tipo numérico
numericas = X_train.drop(columns=["median_income", "ocean_proximity"])
numericas.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households
7061,-118.02,33.93,35.0,2400.0,398.0,1218.0,408.0
14689,-117.09,32.79,20.0,2183.0,534.0,999.0,496.0
17323,-120.14,34.59,24.0,1601.0,282.0,731.0,285.0


In [7]:
# Variable "categorica" que contiene la variable cualitativa ocean_proximity
categorica = X_train.select_dtypes(include=("object"))
categorica.head(3)

,ocean_proximity
7061,<1H OCEAN
14689,NEAR OCEAN
17323,NEAR OCEAN


### Pipeline de Transformación

El primer paso para crear un Pipeline es definir cada tipo de transformador, generalmente se crean diferentes transformadores para cada tipo de variable pero en este caso se va a crear un transformador para los datos que contienen las variables `numéricas`, el cual aplica un StandardScaler para escalar los datos y un SimpleImputer para completar los valores faltantes.

In [8]:
# Se importan las librerías que se van a utilizar

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Se crea el pipeline con las transformaciones para las variables numéricas

num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

Antes de seguir, se creará una clase de sklearn para que el transformador logarítmico que se piensar aplicar sobre la variable `median_income` funcione sin problemas dentro de un Pipeline.

Para crear una clase de sklearn es importante llamar a `TransformerMixin` y `BaseEstimator`. Con el primero se obtiene tres métodos que son muy necesarios: `fit()`, `transform()`y `fit_transform`, con el segundo se obtienen dos métodos adicionales que son: `get_params()`y `set_params()`, los cuales serán útiles para futuros ajustes.

In [9]:
# Se crea la clase para escalar los datos con log(x):

from sklearn.base import BaseEstimator, TransformerMixin

class LogScaler(BaseEstimator, TransformerMixin):
    def __init__(self, add_log_scaler = True, feature_names=None):
        self.add_log_scaler = add_log_scaler
        self.feature_names = feature_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        if self.feature_names is None:
            X = np.log(X)
        else:
            X = np.log(X[self.feature_names])
        return X

Ya se tiene el Pipeline con las transformaciones para las variables numéricas, sin embargo, se quiere también aplicar OneHotEncoder() sobre la variable categórica y una transformación logarítmica sobre la variable continua. Dichas transformaciones se podrían hacer por separado pero sería más cómodo tener un sólo transformador que pueda aplicar cada una las transformaciones definidas anteriormente a cada columna, esto se puede hacer con la clase `ColumnTransformer` de sklearn, de acuerdo a los siguientes pasos:

In [10]:
# Las variables se deben converitr en listas para que funcionen dentro del ColumnTransformer

num_features= list(numericas)
cont_feature= list(continua_df)
cat_feature = list(categorica) 

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Se crea el ColumnTransformer que contendrá todas las transformaciones

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_features), # Aplica sobre las variables numéricas las transformaciones contenidas en el Pipeline
    ("continua", LogScaler(), cont_feature), # Aplica sobre la variable continua la transformación log con la clase que se había creado
    ("categorica", OneHotEncoder(sparse=False,), cat_feature)]) # Aplica sobre la variable categórica OneHotEnconder 

In [12]:
X_train_transformadas = full_pipeline.fit_transform(X_train)

In [13]:
X_train_transformadas

array([[ 0.78093406, -0.80568191,  0.50935748, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.24526986, -1.33947268, -0.67987313, ...,  0.        ,
         0.        ,  1.        ],
       [-0.27755183, -0.49664515, -0.36274497, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.60119118, -0.75885816,  0.58863952, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18625198,  0.90338501, -1.07628333, ...,  0.        ,
         0.        ,  0.        ],
       [-1.41592345,  0.99235014,  1.85715216, ...,  0.        ,
         1.        ,  0.        ]])

In [15]:
X_train_transformadas.shape

(14448, 13)

### También, se puede hacer un pipeline que incluya el preprocesador y el modelo:

In [14]:
from sklearn.linear_model import LinearRegression

# Se construye el pipeline con el preprocesador y un modelo de regresión lineal

pipe_modelo = Pipeline([('preprocessor', full_pipeline),
                        ('modelo', LinearRegression())])

# Se entrena el modelo
pipe_modelo.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'total_rooms',
                                                   'total_bedrooms',
                                                   'population',
                                                   'households']),
                                                 ('continua', LogScaler(),
                                                  ['median_income']),
    